# Acquiring and Cleaning Historical Price Data for Bitcoin and other Cryptocurrencies

__Niklas Gutheil__<br>

__2022-03-01__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import bitfinex
import time
import datetime
import plotly.express as px


## Table of Contents:

## Introduction

To begin any data science project, I will have to acquire my data. For my purposes I will connect to the Bitfinex API and download Bitcoins Historical Price Data in 5-minute intervals, which I can later aggregate to any other timeframe I wish to have. 

Bitfinex also has a custom python library that lets you connect to their public API, which I have installed into my environment using: <br> `pip install bitfinex_tencars`

Since this is a public API I won't need to generate an API key, but this comes with limitations such as only being able to make 1 request per second, and the amount of results allowed to be returned for every request is 10,000.

The basic information the request needs are: 
- `pair`: The trading pair ex: BTCUSD
- `time_interval`: What time resolution you want the data in ex: 1 minute, 1 hour, 1 day etc.
- `t_start`: Start date of requested historical price data
- `t_stop`: End date of requested historical price data <br><br>

I will download the data in 5-minute intervals as 1-minute intervals aren't necessary for the intended purpose of showcasing algorithms, and 1-minute time intervals would create far too many data entries. I will also download all the historical data from __July 22nd, 2014 to January 1st, 2022__. The reason I'm choosing July 22nd, 2014 as the starting point is that before this time trading activity looked very different than today. The market conditions before then were more akin to penny stocks and would thus behave differently than high-volume assets. In addition, I need a 463 days previous price data to calculate some of the features later on, and Bitfinex data only begins on April 15th, 2013. So technically I will be downloading the starting from April 15th, 2013 but only the data from July 22nd, 2014 and onward will be used for modelling.<br><br>

I start by defining what currency pair I want to download, the time intervals the data should come in, and the starting/end date for the data.


In [2]:
pair = 'BTCUSD'

time_interval = '5m'    # 5-minute time interval

t_start = datetime.datetime(2013, 4, 15, 0, 0)     # April 15th, 2013
t_start = time.mktime(t_start.timetuple()) * 1000    # convert starting date to milliseconds since unix epoch

t_stop = datetime.datetime(2022, 4, 5, 0, 0)    # January 1st, 2022
t_stop = time.mktime(t_stop.timetuple()) * 1000


Now I have to build a function that will collect all of our chunks of data and combine them into a single DataFrame. Since I'm using the public REST API, we are limited to 10,000 entries returned per request, and 1 request per second. <br><br>

The function will take in the above parameters and 1 additional parameter `s_inInterval` that will store how many seconds the `time_interval` variable is made up of. This will be important for calculating how many times to call the API to get the full data. <br><br>

The function will return the combined data from all API request.


In [3]:
def fetch_data(start, stop, symbol, interval, s_inInterval):
    limit = 10000    # We want the maximum of 10000 data points

    api = bitfinex.bitfinex_v2.api_v2() # Create API instance
    
    interval_milli = s_inInterval * 1000 # turn our seconds in interval to milliseconds
    step = interval_milli * limit # our step size (time interval for each request) will be 10,000 times the single time interval we want our data in
    data = []

    total_steps = (stop-start)/interval_milli # total number of requests we will have to make
    
    while total_steps > 0:
        if total_steps < limit: # recalculating ending steps
            step = total_steps * interval_milli

        end = start + step # define endpoint for this request
        
        data += api.candles(symbol=symbol, interval=interval, limit=limit, start=start, end=end)
        print(pd.to_datetime(start, unit='ms'), pd.to_datetime(end, unit='ms'), "steps left:", total_steps)
        
        start = start + step # update new start point for next request
        total_steps -= limit # update total_steps left
        
        time.sleep(1.5) #sleep for 1.5 seconds to make sure we dont time out the API
    return data

Now I can fetch the data using the variables defined earlier, as well as defining the `s_inInterval` which will be 300 as there are that many seconds in 5 minutes.

In [4]:
# this cell will take a few minutes to run
s_inInterval = 300

result = fetch_data(t_start, t_stop, pair, time_interval, s_inInterval)
names = ['Date', 'Open', 'Close', 'High', 'Low', 'Volume']
df = pd.DataFrame(result, columns=names)


No keys, only access to public API functions
2013-04-15 04:00:00 2013-05-19 21:20:00 steps left: 943776.0
2013-05-19 21:20:00 2013-06-23 14:40:00 steps left: 933776.0
2013-06-23 14:40:00 2013-07-28 08:00:00 steps left: 923776.0
2013-07-28 08:00:00 2013-09-01 01:20:00 steps left: 913776.0
2013-09-01 01:20:00 2013-10-05 18:40:00 steps left: 903776.0
2013-10-05 18:40:00 2013-11-09 12:00:00 steps left: 893776.0
2013-11-09 12:00:00 2013-12-14 05:20:00 steps left: 883776.0
2013-12-14 05:20:00 2014-01-17 22:40:00 steps left: 873776.0
2014-01-17 22:40:00 2014-02-21 16:00:00 steps left: 863776.0
2014-02-21 16:00:00 2014-03-28 09:20:00 steps left: 853776.0
2014-03-28 09:20:00 2014-05-02 02:40:00 steps left: 843776.0
2014-05-02 02:40:00 2014-06-05 20:00:00 steps left: 833776.0
2014-06-05 20:00:00 2014-07-10 13:20:00 steps left: 823776.0
2014-07-10 13:20:00 2014-08-14 06:40:00 steps left: 813776.0
2014-08-14 06:40:00 2014-09-18 00:00:00 steps left: 803776.0
2014-09-18 00:00:00 2014-10-22 17:20:00 

I'll inspect the data to make sure I have the expected columns.

In [5]:
df.head(5)

,Date,Open,Close,High,Low,Volume
0,1368997500000,116.00,116.00,116.00,116.00,0.979991
1,1368997200000,116.11,116.00,116.11,116.00,18.000000
2,1368995700000,116.46,116.46,116.46,116.46,10.000000
3,1368995400000,116.10,116.10,116.10,116.10,1.340986
4,1368995100000,117.95,117.95,117.95,117.95,1.000000


The data looks to be in the form that I'm looking for. The columns can be interpreted as follows:
- __Date__: Time in Seconds since the Unix Epoch
- __Open__: The price of Bitcoin in USD at the beginning of our 5-minute interval
- __Close__: The price of Bitcoin in USD at the end of our 5-minute interval
- __High__: The highest price of Bitcoin in USD during our 5-minute interval
- __Low__: The lowest price of Bitcoin in USD during our 5-minute interval
- __Volume__: The amount of Bitcoin bought and sold in our 5-minute interval


Now I can take a look at duplicate values and decide if I want to drop them.

In [6]:
print(f'{df.duplicated().sum()} Duplicate Entries')
print(f'{np.round(df.duplicated().sum()/df.shape[0]*100, decimals = 2)}% of Entries are Duplicates')

65 Duplicate Entries
0.01% of Entries are Duplicates


In [7]:
df[df.duplicated(keep = False)].head(15)

,Date,Open,Close,High,Low,Volume
7091,1371998400000,100.50,100.50,100.50,100.50000,0.400000
19452,1371998400000,100.50,100.50,100.50,100.50000,0.400000
19453,1377998400000,127.27,128.43,128.92,126.81000,104.252037
29212,1377998400000,127.27,128.43,128.92,126.81000,104.252037
29213,1383998400000,341.90,342.80,342.80,341.49000,62.103869
36391,1386998400000,873.00,873.00,873.00,872.99000,6.478167
45901,1383998400000,341.90,342.80,342.80,341.49000,62.103869
45902,1389998400000,796.00,799.99,800.00,796.00000,29.689615
55527,1386998400000,873.00,873.00,873.00,872.99000,6.478167
55528,1392998400000,565.00,566.00,566.00,564.76001,57.726278


The entries appear to be perfect duplicates as they share the same timestamp, and all other corresponding values. They also only make up a tiny fraction of the total data, so I can safely drop them. One issue this might cause later is that I now have missing values for some timestamps. I will investigate this later in the notebook.

In [8]:
df.drop_duplicates(inplace=True)

I can also see that the Date column is showing the seconds since the Unix epoch. I want to convert this column into a DateTime format so I can use this dataset for timeseries specific analysis. Additionally, I'll set the index to be the Date column, as each timestamp is unique.

In [9]:
df['Date'] = pd.to_datetime(df['Date'], unit='ms')
df.set_index('Date', inplace=True)

In [10]:
df.head(10)

,Open,Close,High,Low,Volume
Date,,,,,
2013-05-19 21:05:00,116.00,116.00,116.00,116.00,0.979991
2013-05-19 21:00:00,116.11,116.00,116.11,116.00,18.000000
2013-05-19 20:35:00,116.46,116.46,116.46,116.46,10.000000
2013-05-19 20:30:00,116.10,116.10,116.10,116.10,1.340986
2013-05-19 20:25:00,117.95,117.95,117.95,117.95,1.000000
2013-05-19 19:15:00,116.97,117.00,117.00,116.52,65.353146
2013-05-19 19:10:00,116.94,116.94,116.94,116.94,9.000000
2013-05-19 19:05:00,116.84,116.84,116.89,116.84,7.541986
2013-05-19 19:00:00,116.78,117.00,117.00,116.78,12.000000


The columns are looking much better now, but another problem I might have is missing entries for a specific time interval. Let's explore if and how many are missing.

In [11]:
first_time = df.index.min()
last_time = df.index.max()
print(first_time, last_time)

2013-04-15 04:00:00 2022-04-05 04:00:00


In [12]:
full_range = pd.date_range(start=first_time, end=last_time, freq="5min")
differences = full_range.difference(df.index)

In [13]:
print(f"{100*round(len(differences)/df.shape[0], 6)}% of intervals missing")

4.7973% of intervals missing


I can see that only 4.95% of the entries are missing, which is good, as imputing entries won't lower the results by much. First I'll check if there are any NULL values in the dataset.

In [14]:
df.isna().sum()

Open      0
Close     0
High      0
Low       0
Volume    0
dtype: int64

No NULL values is great to see! <br>
I should also make sure none of the values are 0, in case Bitfinex imputed null values with 0.

In [15]:
display(np.where(df['Volume'] == 0))
display(np.where(df['Open'] == 0))
display(np.where(df['Close'] == 0))
display(np.where(df['High'] == 0))
display(np.where(df['Low'] == 0))

(array([], dtype=int64),)

(array([], dtype=int64),)

(array([], dtype=int64),)

(array([], dtype=int64),)

(array([], dtype=int64),)

None of the values are 0 so I'm now confident I have real values for the existing entries.

The next step is to create entries for the missing date ranges, and then impute those null values with some kind of value. I could use forwardfilling to simply impute the last known value, but using the mean of the last previous and next known value seems like a more appropriate choice as I'm dealing with price information, which has to rise through a given set of values. <br>
I will set the method to 'time' for interpolate as this will also account for missing entries of 2 or more in a row. Instead of imputing both consecutive missing time intervals with the same number, it will split this increase over the amount of missing rows.

In [16]:
df_clean = df.reindex(full_range)

In [17]:
df_clean = df_clean.interpolate(method='time')

Let's check if there are any missing time intervals or NULL values.

In [18]:
first_time = df_clean.index.min()
last_time = df_clean.index.max()

full_range = pd.date_range(start=first_time, end=last_time, freq="5min")
differences = full_range.difference(df_clean.index)

df.sort_index(inplace=True) # make sure all the entries are in order of date

print(f"{100*round(len(differences)/df_clean.shape[0], 6)}% of intervals missing")

display(df.isna().sum())

print(df_clean.shape)

0.0% of intervals missing


Open      0
Close     0
High      0
Low       0
Volume    0
dtype: int64

(943777, 5)


I now have a fully cleaned dataset with no missing intervals, no null values and 916,717 entries! Let's view Bitcoins historical price data from April 15th, 2013 to January 1st, 2022 in a graph. I will resample the data first into 1-day time intervals, as otherwise the notebook becomes 76MB all from one graph.

In [26]:
graphing_df = df.resample('D').mean()
fig = px.line(graphing_df, x=graphing_df.index, y=graphing_df.Open,)

# axis labels and title
fig.update_layout(
    yaxis_title="Price in USD", 
    xaxis_title="Date",
    legend_title="", 
    title = "Price of Bitcoin from April 2013 - January 2022",
    yaxis_tickprefix = '$', 
    yaxis_tickformat = ',.2f'
)

# activate slider
fig.update_xaxes(rangeslider_visible=True)
fig.update_yaxes(type = 'log')

fig.show()

Lastly, let's save this data as a csv file for easier later use.

In [20]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 943777 entries, 2013-04-15 04:00:00 to 2022-04-05 04:00:00
Freq: 5T
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Open    943777 non-null  float64
 1   Close   943777 non-null  float64
 2   High    943777 non-null  float64
 3   Low     943777 non-null  float64
 4   Volume  943777 non-null  float64
dtypes: float64(5)
memory usage: 75.5 MB


In [27]:
df_clean.to_csv(f"./data/{pair}_{time_interval} historical data.csv")

Here I will also resample the dataset into different time intervals for later analysis and feature enginerring. The time intervals will be the 1 Hour and 1 Day. We could resample to 1 Week intervals, but that would leave us with only a few hundred data points, too little for proper analysis in comparison to the other datasets.

In [28]:
df_clean_1H = df_clean.resample('H').mean()
df_clean_1D = df_clean.resample('D').mean()

In [29]:
df_clean_1H.head(2)

,Open,Close,High,Low,Volume
2013-04-15 04:00:00,94.227744,94.229792,94.258976,94.203369,26.687205
2013-04-15 05:00:00,95.690673,95.815542,96.338024,95.396756,26.113038


In [30]:
df_clean_1D.head(2)

,Open,Close,High,Low,Volume
2013-04-15,93.343552,93.196398,93.779854,92.732950,70.253082
2013-04-16,66.376708,66.283507,67.081630,65.280107,92.148519


I will also save these dataframes as csv files for later analysis and feature engineering. 

In [31]:
df_clean_1H.to_csv(f"./data/{pair}_1H Historical Data.csv")
df_clean_1D.to_csv(f"./data/{pair}_1D Historical Data.csv ")

That brings me to the end of this notebook. With slight modification of the initial code, any time interval of any Cryptocurrency Pair's Historical Price Data on Bitfinex can be acquired. The notebook will also clean and output a .csv file for the original time interval, as well as a 1 Hour and 1 Day interval. Important to note is that the original time interval chosen needs to be less than 1 Hour to ensure no NULL entries when resampling.